In [12]:
from __future__ import print_function

import numpy as np

from sklearn import __version__ as sklearn_version
print('Sklearn version:', sklearn_version)

Sklearn version: 0.19.0


# The data

The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics split in two subsets: one for training (or development) and the other one for testing (or for performance evaluation). The split between the train and test set is based upon a messages posted before and after a specific date.


In [13]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

twenty_train = fetch_20newsgroups(subset='train',
                 remove=('headers', 'footers', 'quotes'),
                 categories=categories, shuffle=True, random_state=42)

twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [14]:
# Sample data
print(twenty_train.data[1])
print('---------------')
print('Target: ', twenty_train.target[0])


Hi,

	I have a problem, I hope some of the 'gurus' can help me solve.

	Background of the problem:
	I have a rectangular mesh in the uv domain, i.e  the mesh is a 
	mapping of a 3d Bezier patch into 2d. The area in this domain
	which is inside a trimming loop had to be rendered. The trimming
	loop is a set of 2d Bezier curve segments.
	For the sake of notation: the mesh is made up of cells.

	My problem is this :
	The trimming area has to be split up into individual smaller
	cells bounded by the trimming curve segments. If a cell
	is wholly inside the area...then it is output as a whole ,
	else it is trivially rejected. 

	Does any body know how thiss can be done, or is there any algo. 
	somewhere for doing this.

	Any help would be appreciated.

	Thanks, 
	Ani.
---------------
Target:  1


In [15]:
# Text preprocessing, tokenizing and filtering of stopwords

from sklearn.feature_extraction.text import CountVectorizer
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=5000,
                                stop_words='english')
X_train_counts = tf_vectorizer.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 5000)

In [16]:
print(X_train_counts[0,:])
print(X_train_counts[:,0])


  (0, 2866)	1
  (0, 238)	1
  (0, 4522)	1
  (0, 2058)	1
  (0, 1123)	1
  (0, 3867)	1
  (0, 1543)	1
  (0, 3385)	1
  (0, 2197)	1
  (0, 1094)	1
  (0, 2643)	1
  (0, 1865)	1
  (0, 2237)	1
  (0, 1795)	2
  (0, 4520)	1
  (0, 2251)	1
  (0, 1090)	1
  (0, 4744)	1
  (0, 3276)	1
  (0, 357)	1
  (0, 3273)	1
  (0, 4299)	1
  (0, 4869)	1
  (0, 2014)	1
  (0, 2550)	1
  (0, 1445)	1
  (232, 0)	2
  (272, 0)	1
  (282, 0)	1
  (400, 0)	1
  (433, 0)	2
  (581, 0)	2
  (588, 0)	1
  (766, 0)	1
  (768, 0)	2
  (837, 0)	3
  (844, 0)	1
  (859, 0)	1
  (880, 0)	1
  (1030, 0)	1
  (1056, 0)	6
  (1057, 0)	2
  (1263, 0)	1
  (1475, 0)	1
  (1665, 0)	16
  (1795, 0)	1
  (1802, 0)	1
  (1833, 0)	1
  (1890, 0)	2
  (2069, 0)	1
  (2144, 0)	1


In [17]:
#From occurrences to frequencies
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(X_train_counts)
X_train_tf = tfidf_transformer.transform(X_train_counts)
X_train_tf.shape

(2257, 5000)

In [18]:
print(X_train_tf[0,:])
print(X_train_tf[:,0])


  (0, 1445)	0.0998496101737
  (0, 2550)	0.0920875619201
  (0, 2014)	0.10905059472
  (0, 4869)	0.112409159775
  (0, 4299)	0.172232378831
  (0, 3273)	0.189497984618
  (0, 357)	0.196147304589
  (0, 3276)	0.239358101611
  (0, 4744)	0.242697172074
  (0, 1090)	0.185367646905
  (0, 2251)	0.281517460204
  (0, 4520)	0.239358101611
  (0, 1795)	0.326673936513
  (0, 2237)	0.217882788689
  (0, 1865)	0.182356290661
  (0, 2643)	0.0944312658437
  (0, 1094)	0.250397930473
  (0, 2197)	0.225991796704
  (0, 3385)	0.272954303671
  (0, 1543)	0.163780615995
  (0, 3867)	0.165608347231
  (0, 1123)	0.157610927262
  (0, 2058)	0.144807482284
  (0, 4522)	0.126533637604
  (0, 238)	0.170069829145
  (0, 2866)	0.190380209723
  (232, 0)	0.162673301572
  (272, 0)	0.0396045882998
  (282, 0)	0.0830143471237
  (400, 0)	0.00527736458963
  (433, 0)	0.00596499373539
  (581, 0)	0.150704657006
  (588, 0)	0.154296833105
  (766, 0)	0.126956846998
  (768, 0)	0.0117078298784
  (837, 0)	0.334685959895
  (844, 0)	0.207167396703
  (85

## First basic model 

In [19]:
from sklearn.naive_bayes import MultinomialNB

# Define and fit in one line
clf = MultinomialNB().fit(X_train_tf, twenty_train.target)


In [20]:
#Score test data

# Read test data
twenty_test = fetch_20newsgroups(subset='test',
                 remove=('headers', 'footers', 'quotes'),
                 categories=categories, shuffle=True, random_state=42)

# Transform text to counts
X_test_counts = tf_vectorizer.transform(twenty_test.data)

# tf-idf transformation
X_test_tf = tfidf_transformer.transform(X_test_counts)

# Prediction
predicted = clf.predict(X_test_tf)

# Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy test: ', accuracy_score(twenty_test.target, predicted))


Accuracy test:  0.798934753662


In [21]:
# Score 2 new docs
docs_new = ['God loves GPU', 'OpenGL on the GPU is fast']

def score_text(text_list):
    '''
    Score function
    '''
    X_new_counts = tf_vectorizer.transform(docs_new)
    X_new_tfidf = tfidf_transformer.transform(X_new_counts)
    predicted = clf.predict(X_new_tfidf)
    return predicted


for doc, category in zip(docs_new, score_text(docs_new)):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'God loves GPU' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics
